In [ ]:
!pip install rdkit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Defining the directory path
#folder_path = "/content/drive/MyDrive/project2"
# Create the folder if it doesn't exist
if not os.path.exists("/content/drive/MyDrive/project2"):
    os.makedirs("/content/drive/MyDrive/project2")
if not os.path.exists("/content/drive/MyDrive/project2/x_train"):
    os.makedirs("/content/drive/MyDrive/project2/x_train")
if not os.path.exists("/content/drive/MyDrive/project2/x_test"):
    os.makedirs("/content/drive/MyDrive/project2/x_test")
if not os.path.exists("/content/drive/MyDrive/project2/external_filtered"):
    os.makedirs("/content/drive/MyDrive/project2/external_filtered")




from rdkit import Chem
from rdkit.Chem import AllChem
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
def classifier(file,clf,CA,K,auc,parameter):
    listt=[]

    smiles1=[]
    smiles2=[]
    w=pd.read_csv(file,index_col=0)
    #H=b["Molecule ChEMBL ID"]
    #print(w)
    o=w["Smiles"]
    for sno,i in enumerate(o):
        mol=Chem.MolFromSmiles(i)
        mp=AllChem.GetMorganFingerprintAsBitVect(mol,radius=2,nBits=1024)
        listt.append([sno,list(mp)])
    #print(listt)
    data= w.to_dict(orient="split")['data']
    print(w.to_dict(orient="split")['columns'])
    x=listt
    y=w["Activity"]
    l=parameter
    e=[]
    f=[]
    f.append(["iteration","Parameter","AUC","TPR_0","FPR_0","PRES_0","TPR_1","FPR_1","PRES_1","Classifier"])
    for j in range(K):

        for i in l:

            if type(i)==int:
                X_train,test_X,y_train,test_lab=train_test_split(x,y,test_size=0.2,random_state=None)
                x_train=[i[1] for i in X_train]
                test_x=[i[1] for i in test_X]

                clff=clf(n_neighbors=i).fit(x_train,y_train)


            else:
                X_train,test_X,y_train,test_lab=train_test_split(x,y,test_size=i,random_state=None)
                x_train=[i[1] for i in X_train]
                test_x=[i[1] for i in test_X]
                clff=clf.fit(x_train,y_train)
                #print(len(X_train))
            pred=clff.predict(test_x)
            y_true=test_lab
            y_pred=pred
            #ARI=adjusted_rand_score(pred,test_lab)
            FP=np.logical_and(y_true !=y_pred,y_pred ==0).sum()
            FN=np.logical_and(y_true !=y_pred,y_pred !=0).sum()
            TP=np.logical_and(y_true ==y_pred,y_true ==0).sum()
            TN=np.logical_and(y_true ==y_pred,y_true !=0).sum()
            TPR=TP/(TP+FN)
            FPR=FP/(TN+FP)
            PRES=TP/(TP+FP)
            REC=TP/(TP+FN)
            FP=np.logical_and(y_true !=y_pred,y_pred ==1).sum()
            FN=np.logical_and(y_true !=y_pred,y_pred !=1).sum()
            TP=np.logical_and(y_true ==y_pred,y_true ==1).sum()
            TN=np.logical_and(y_true ==y_pred,y_true !=1).sum()
            TPR_1=TP/(TP+FN)
            FPR_1=FP/(TN+FP)
            PRES_1=TP/(TP+FP)
            AUC=roc_auc_score(test_lab,pred)
            if AUC>=auc:
                print([j,i,AUC,TPR,FPR,PRES,TPR_1,FPR_1,PRES_1,CA])
                f.append([j,i,AUC,TPR,FPR,PRES,TPR_1,FPR_1,PRES_1,CA])
                train_lx=[w.to_dict(orient="split")['columns']]+[data[i[0]] for i in X_train]
                test_lx=[w.to_dict(orient="split")['columns']]+[data[i[0]] for i in test_X]
                #print([train_lx.count(i) for i in train_lx],[test_lx.count(i) for i in test_lx])
                g=pd.DataFrame(train_lx)
                h=pd.DataFrame(test_lx)
                g.to_csv(f"/content/drive/MyDrive/project2/x_train/x_train_{CA}{i}{j}.csv",header=False,index=False)
                h.to_csv(f"/content/drive/MyDrive/project2/x_test/x_test_{CA}{i}{j}.csv",header=False,index=False)
    F=pd.DataFrame(f)
    #print(F)
    F.to_csv(f"/content/drive/MyDrive/project2/AUC_{CA}_{K}.csv",header=False,index=False)
    print(f"/content/drive/MyDrive/project2/AUC_{CA}_{K}.csv")


    external=pd.read_csv("/content/tdp_external.csv")
    external1=external["Smiles"]
    for i in external1:
        mole=Chem.MolFromSmiles(i)
        mp=AllChem.GetMorganFingerprintAsBitVect(mole,radius=2,nBits=1024)
        smiles1.append(list(mp))
    x_test1= smiles1
    y_test1=external["Activity"]

    # Replacing 'folder_path' with the path to the folder containing CSV files
    folder_path = "/content/drive/MyDrive/project2/x_train"

    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Iterating  through each CSV file and performing actions individually
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        # Loading the CSV file into a DataFrame
        sorted_file = pd.read_csv(file_path)
        o1=sorted_file["Smiles"]
        listt1=[]
        for i in o1:
            mo=Chem.MolFromSmiles(i)
            mp=AllChem.GetMorganFingerprintAsBitVect(mo,radius=2,nBits=1024)
            listt1.append(list(mp))
        f=[]
        f.append(["AUC","TPR_0","FPR_0","PRES_0","TPR_1","FPR_1","PRES_1","Classifier","file_path"])
        x_train1=listt1
        print(len(x_train1))
        y_train1=sorted_file["Activity"]
        print(len(y_train1))
        clff=clf.fit(x_train1,y_train1)
        pred=clff.predict(x_test1)
        y_true=y_test1
        y_pred=pred
        FP=np.logical_and(y_true !=y_pred,y_pred ==0).sum()
        FN=np.logical_and(y_true !=y_pred,y_pred !=0).sum()
        TP=np.logical_and(y_true ==y_pred,y_true ==0).sum()
        TN=np.logical_and(y_true ==y_pred,y_true !=0).sum()
        TPR=TP/(TP+FN)
        FPR=FP/(TN+FP)
        PRES=TP/(TP+FP)
        REC=TP/(TP+FN)
        FP=np.logical_and(y_true !=y_pred,y_pred ==1).sum()
        FN=np.logical_and(y_true !=y_pred,y_pred !=1).sum()
        TP=np.logical_and(y_true ==y_pred,y_true ==1).sum()
        TN=np.logical_and(y_true ==y_pred,y_true !=1).sum()
        TPR_1=TP/(TP+FN)
        FPR_1=FP/(TN+FP)
        PRES_1=TP/(TP+FP)
        AUC=roc_auc_score(y_test1,pred)
        if AUC>=auc:
            source_file = file_path
            destination_file = '/content/drive/MyDrive/project2/external_filtered'
            shutil.copy(source_file, destination_file)

#blind
    blindset=pd.read_csv("/content/zinc-set (2).csv")
    external1=blindset["Smiles"]
    for i in external1:
        mole=Chem.MolFromSmiles(i)
        mp=AllChem.GetMorganFingerprintAsBitVect(mole,radius=2,nBits=1024)
        smiles2.append(list(mp))
    x_test2= smiles2

    folderpath =destination_file

    csv_files = [file for file in os.listdir(folderpath) if file.endswith('.csv')]

    for csv_file in csv_files:
        file_path = os.path.join(folderpath, csv_file)
        sortedfile = pd.read_csv(file_path)
        o2=sortedfile["Smiles"]
        blind=[]
        for i in o2:
            mo=Chem.MolFromSmiles(i)
            mp=AllChem.GetMorganFingerprintAsBitVect(mo,radius=2,nBits=1024)
            blind.append(list(mp))

        x_train2=blind
        y_train2=sortedfile["Activity"]
        clff=clf.fit(x_train2,y_train2)
        pred=clff.predict(x_test2)
        print(pred)

classifier("/content/TDP-Activity-Sorted1.csv",RandomForestClassifier(),"RF",500,0.85,[0.2])
classifier("/content/TDP-Activity-Sorted1.csv",GaussianProcessClassifier(),"GaussianProcessClassifier",1000,0.8,[0.2])
classifier("/content/TDP-Activity-Sorted1.csv",RidgeClassifier(),"Ridgeclassifier",1000,0.8,[0.2])
classifier("/content/TDP-Activity-Sorted1.csv",BernoulliNB(),"BernoulliNB",1000,0.8,[0.2])
classifier("/content/TDP-Activity-Sorted1.csv",RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1]),"RidgeClassifierCV",1000,0.8,[0.2])
classifier("/content/TDP-Activity-Sorted1.csv",Perceptron(tol=1e-3, random_state=0),"Perceptron",1000,0.8,[0.2])